In [1]:
from cosecha_colectiva import miscelaneous as m
from cosecha_colectiva import config as config
m.try_find_conf_file()


from cosecha_colectiva import miscelaneous as m

import logging
import pathlib
from datetime import datetime
import datajoint as dj
import time
import numpy as np

import pandas as pd


Local configuration file found !!, no need to run the configuration (unless configuration has changed)


In [35]:
def number_to_excel_column(n):
    result = ''
    while n > 0:
        remainder = (n - 1) % 26
        result = chr(ord('A') + remainder) + result
        n = (n - 1) // 26
    return result

In [2]:
cosecha_db = dj.create_virtual_module(config.db_name, config.db_name)

[2025-05-05 14:50:39,779][INFO]: Connecting root@junction.proxy.rlwy.net:19994
[2025-05-05 14:50:41,113][INFO]: Connected root@junction.proxy.rlwy.net:19994


In [ ]:
ogsd = cosecha_db.OldGruposStatsDashboard()
table_attributes = pd.DataFrame.from_dict(ogsd.original_heading.attributes,orient='index')
this_dtypes = table_attributes.dtype.to_dict()
this_cols = table_attributes.index.to_list()

columns_read = 'A:'+number_to_excel_column(len(this_cols))
columns_read

lo

In [ ]:
for i in range(0, len(config.list_viejos_grupos_sheets_id)):
    print(i)
    this_id = config.list_viejos_grupos_sheets_id[i]
    this_data = m.read_excel_sheet_by_id(this_id, 'AutoStats')

    base_url = 'https://docs.google.com/spreadsheets/d/'
    suffix = '/export?format=xlsx'
    url = base_url+this_id+suffix

    df_data = pd.read_excel(url,
                          sheet_name='AutoStats',
                          skiprows=30, nrows=1, names=this_cols, header=None, dtype=this_dtypes
                  )
    df_data = df_data.replace(np.nan, None)
    dict_data = df_data.to_dict('records')
    ogsd.insert(dict_data, skip_duplicates=True)
    time.sleep(0.1)

    if i==0:
        break

0


In [64]:
num_socios_grupo = [] 
Grupos_Names = []
num_sesiones_grupo = []
for i in range(0, len(config.list_viejos_grupos_sheets_id)):
    print(i)
    this_id = config.list_viejos_grupos_sheets_id[i]
    this_data = m.read_excel_sheet_by_id(this_id, 'AutoStats')

    base_url = 'https://docs.google.com/spreadsheets/d/'
    suffix = '/export?format=xlsx'
    url = base_url+this_id+suffix

    data_num_socios = pd.read_excel(url,
                          sheet_name='DatosParaStats',
                        nrows=3, usecols='A:B', header=None
                  )
    num_socios_grupo.append(int(data_num_socios.loc[0,1]))
    Grupos_Names.append(data_num_socios.loc[1,1])
    data_num_sesiones = pd.read_excel(url,
                          sheet_name='DatosParaStats',
                          skiprows=59, nrows=1, usecols='A:C', header=None
                  )
    num_sesiones_grupo.append(int(data_num_sesiones.loc[0,2]))

    if i==0:
        break

0


In [65]:
ossd = cosecha_db.OldSociosStatsDashboard()
table_attributes = pd.DataFrame.from_dict(ossd.original_heading.attributes,orient='index')
this_dtypes = table_attributes.dtype.to_dict()
this_cols = table_attributes.index.to_list()

columns_read = 'A:'+number_to_excel_column(len(this_cols))
columns_read

'A:BI'

In [ ]:
for i in range(0, len(config.list_viejos_grupos_sheets_id)):
    print(i)
    print(Grupos_Names[i])
    print('num_socios_grupo[i]', num_socios_grupo[i])
    print('num_sesiones_grupo[i]', num_sesiones_grupo[i])
    this_id = config.list_viejos_grupos_sheets_id[i]
    this_data = m.read_excel_sheet_by_id(this_id, 'AutoStats')

    base_url = 'https://docs.google.com/spreadsheets/d/'
    suffix = '/export?format=xlsx'
    url = base_url+this_id+suffix

    df_data = pd.read_excel(url,
                          sheet_name='AutoStats',
                          skiprows=1, nrows=num_socios_grupo[i], names=this_cols, usecols=columns_read, dtype=this_dtypes, header=None
                  )
    df_data = df_data.replace(np.nan, None)
    dict_data = df_data.to_dict('records')
    ossd.insert(dict_data, skip_duplicates=True)
    time.sleep(0.1)

    if i==0:
        break

0
La Luz
num_socios_grupo[i] 12
num_sesiones_grupo[i] 92


DuplicateError: ("Duplicate entry '1-1' for key 'old_socios_stats_dashboard.PRIMARY'", 'To ignore duplicate entries in insert, set skip_duplicates=True')

In [ ]:
ossd = cosecha_db.OldSociosStatsDashboard()
table_attributes = pd.DataFrame.from_dict(ossd.original_heading.attributes,orient='index')
this_dtypes = table_attributes.dtype.to_dict()
this_cols = table_attributes.index.to_list()

columns_read = 'A:'+number_to_excel_column(len(this_cols))
columns_read

In [ ]:
for i in range(0, len(config.list_viejos_grupos_sheets_id)):
    print(i)
    print(Grupos_Names[i])
    print('num_socios_grupo[i]', num_socios_grupo[i])
    print('num_sesiones_grupo[i]', num_sesiones_grupo[i])
    this_id = config.list_viejos_grupos_sheets_id[i]
    this_data = m.read_excel_sheet_by_id(this_id, 'AutoStats')

    base_url = 'https://docs.google.com/spreadsheets/d/'
    suffix = '/export?format=xlsx'
    url = base_url+this_id+suffix

    df_data = pd.read_excel(url,
                          sheet_name='AutoStatsSesiones',
                          skiprows=200, nrows=num_sesiones_grupo[i], names=this_cols, usecols=columns_read, dtype=this_dtypes, header=None
                  )
    df_data = df_data.replace(np.nan, None)
    dict_data = df_data.to_dict('records')
    ossd.insert(dict_data, skip_duplicates=True)
    time.sleep(0.1)

    if i==0:
        break

,Grupo_id,Nombre_grupo,num_socio_grupo,socio_id,nombres,apellidos,nacionalidad,sexo,Escolaridad,Ocupacion,...,sum_abonos_prestamos,count_abonos_prestamos,sum_compra_acciones,count_compra_acciones,sum_entrega_prestamos,count_entrega_prestamos,sum_pago_multas,count_pago_multas,sum_retiro_acciones,count_retiro_acciones
0,1,La Luz,Socio 1 La Luz,1,Evangelina,None,None,M,None,None,...,25692.0,75,4650.0,70,31348.0,14,155.0,18,0.0,0
1,1,La Luz,Socio 2 La Luz,2,José Dolores,None,None,H,None,None,...,100318.0,71,12020.0,85,121688.0,17,170.0,20,0.0,0
2,1,La Luz,Socio 3 La Luz,3,Lourdes,None,None,M,None,None,...,20720.0,51,14220.0,76,28620.0,16,165.0,17,0.0,0
3,1,La Luz,Socio 4 La Luz,4,Alicia,None,None,M,None,None,...,32500.0,43,11880.0,72,33500.0,12,80.0,10,0.0,0
4,1,La Luz,Socio 5 La Luz,5,Luciana,None,None,M,None,None,...,10500.0,7,6660.0,23,28000.0,5,60.0,6,0.0,0
5,1,La Luz,Socio 6 La Luz,6,Elisabeth,None,None,M,None,None,...,113918.0,77,22620.0,91,108118.0,17,15.0,2,0.0,0
6,1,La Luz,Socio 7 La Luz,7,María Guadalupe,None,None,M,None,None,...,78804.0,75,16140.0,81,88804.0,20,30.0,3,2000.0,1
7,1,La Luz,Socio 8 La Luz,8,Violeta,None,None,M,None,None,...,56834.0,75,5800.0,85,87792.0,15,80.0,10,0.0,0
8,1,La Luz,Socio 9 La Luz,9,Marisol,None,None,M,None,None,...,262060.0,79,132950.0,86,311060.0,20,80.0,10,47000.0,1
9,1,La Luz,Socio 10 La Luz,10,Ana María,None,None,M,None,None,...,137904.0,85,17940.0,87,174404.0,16,40.0,4,0.0,0


In [17]:
df_data['apellidos']

1  La Luz  Socio 1 La Luz   1   Evangelina         None
           Socio 2 La Luz   2   José Dolores       None
           Socio 3 La Luz   3   Lourdes            None
           Socio 4 La Luz   4   Alicia             None
           Socio 5 La Luz   5   Luciana            None
           Socio 6 La Luz   6   Elisabeth          None
           Socio 7 La Luz   7   María Guadalupe    None
           Socio 8 La Luz   8   Violeta            None
           Socio 9 La Luz   9   Marisol            None
           Socio 10 La Luz  10  Ana María          None
           Socio 11 La Luz  11  Brenda             None
           Socio 12 La Luz  12  CEIBAS             None
Name: apellidos, dtype: object

In [11]:
df_data

,Grupo_id,Nombre_grupo,Codigo_grupo,Localidad,Municipio,Estado,CP,Pais,Fecha_reg,Status,...,sum_abonos_prestamos,count_abonos_prestamos,sum_compra_acciones,count_compra_acciones,sum_entrega_prestamos,count_entrega_prestamos,sum_pago_multas,count_pago_multas,sum_retiro_acciones,count_retiro_acciones
0,1,La Luz,1,None,None,Querétaro,None,México,None,1,...,880316.0,702,255030.0,828,1067300.0,163,905.0,105,49000.0,2
